# Adaptive Language Modeling for STEM Education
## Fine-tuning GPT-2 with QLoRA for Mathematical Problem Solving

**Author**: Udit Negi  
**GitHub**: [github.com/uditnegi16](https://github.com/uditnegi16)  
**Date**: 2024-08-25

---

### Project Overview
This notebook demonstrates fine-tuning GPT-2 using QLoRA (Quantized Low-Rank Adaptation) on the arXiv Math Instruct dataset to create an adaptive language model for STEM education applications.

## 1. Environment Setup

In [ ]:
# Install required packages
%pip install transformers datasets peft
%pip install rake-nltk tiktoken bitsandbytes
%pip install accelerate -U

## 2. Data Preparation

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load and preprocess dataset
dataset = load_dataset("Sharathhebbar24/arxiv-math-instruct-50k")
df = dataset['train'].to_pandas()

def structure_data(row):
    conversation = row['prompt']
    user_start = conversation.find('<|user|>') + len('<|user|>')
    assistant_start = conversation.find('<|assistant|>') + len('<|assistant|>')
    question = conversation[user_start:assistant_start].strip()
    answer = conversation[assistant_start:].strip()
    return {"input": f"STEM Question: {question}", "output": answer}

structured_data = df.apply(structure_data, axis=1).tolist()

## 3. Model Configuration

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
import torch

# Model setup with QLoRA
model_name = "gpt2-medium"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True  # Enable 4-bit quantization
)

# LoRA configuration
peft_config = LoraConfig(
    r=8,  # Rank
    lora_alpha=32,
    target_modules=["c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

## 4. Training Setup

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    save_steps=500,
    logging_steps=100,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

## 5. Training Execution

In [ ]:
# Train with progress tracking
print("Starting training...")
trainer.train()

# Save model
model.save_pretrained("./stem-tutor-lora")
print("Training complete!")

## 6. Evaluation

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

# Generate predictions
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

# Calculate accuracy
accuracy = accuracy_score(test_dataset['labels'], preds)
print(f"Test Accuracy: {accuracy:.2%}")

## 7. Deployment Example

In [ ]:
from transformers import pipeline

# Create STEM tutor pipeline
stem_tutor = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0
)

# Example usage
question = "Explain the concept of derivatives in calculus"
response = stem_tutor(f"STEM Question: {question}", max_length=200)
print(response[0]['generated_text'])